# tcga_reader 代码练习

In [1]:
import os
os.chdir('D:/github_project/BioXNet')
os.getcwd()

'D:\\github_project\\BioXNet'

In [2]:
import sys
sys.path.append('D:/github_project/BioXNet/data_loader')
sys.path

['d:\\github_project\\BioXNet\\coding_practice',
 'd:\\anaconda\\envs\\bioxnet\\python311.zip',
 'd:\\anaconda\\envs\\bioxnet\\DLLs',
 'd:\\anaconda\\envs\\bioxnet\\Lib',
 'd:\\anaconda\\envs\\bioxnet',
 '',
 'd:\\anaconda\\envs\\bioxnet\\Lib\\site-packages',
 'd:\\anaconda\\envs\\bioxnet\\Lib\\site-packages\\win32',
 'd:\\anaconda\\envs\\bioxnet\\Lib\\site-packages\\win32\\lib',
 'd:\\anaconda\\envs\\bioxnet\\Lib\\site-packages\\Pythonwin',
 'D:/github_project/BioXNet/data_loader']

In [3]:
import pandas as pd
import numpy as np
import json
from os.path import join, exists
from pathlib import Path
from parse_config import ConfigParser
from sklearn.model_selection import train_test_split, KFold
from sklearn.utils import class_weight

## 读取 .json 文件

In [4]:
# 读取 .json 文件保存为 config 对象
with open('D://github_project//BioXNet//config//tcga_multiclass_nopretrain.json') as f:
    config = json.load(f)

print(type(config))
config

<class 'dict'>


{'name': 'BioXNet_TCGA_MultiClass_NoPretrain_Gene-GDSC-TCGA_classweights',
 'n_gpu': 1,
 'pretrain': None,
 'transfer_directly': False,
 'arch': {'type': 'BioXNet',
  'args': {'n_hidden_layers': 5,
   'direction': 'root_to_leaf',
   'dropout_list': [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
   'sparse': True,
   'add_unk_genes': False,
   'batch_normal': True,
   'use_bias': True,
   'attention': True,
   'sparse_first_layer': True,
   'class_num': 4,
   'n_outputs': 6,
   'shuffle_genes': False}},
 'data_loader': {'type': 'BioXNetDataLoader',
  'args': {'data_source_list': ['TCGA'],
   'input_data_order': ['drug_target', 'mutation', 'cnv', 'methylation'],
   'batch_size': 64,
   'seed': 1,
   'shuffle': True,
   'num_workers': 8,
   'eval_dataset': True,
   'pre_params': None,
   'gene_intersection': True,
   'params': {'data_dir': 'data/',
    'mut_binary': True,
    'selected_genes': None,
    'combine_type': 'intersection',
    'use_coding_genes_only': True,
    'binary_response': False}

设置 TCGAData() 参数

In [5]:
params = config['data_loader']['args']['params']
params

{'data_dir': 'data/',
 'mut_binary': True,
 'selected_genes': None,
 'combine_type': 'intersection',
 'use_coding_genes_only': True,
 'binary_response': False}

In [6]:
data_path = params['data_dir']
data_list = config['data_loader']['args']['data_source_list']

config = ConfigParser(config)
logger = config.get_logger('train')

mut_binry = params['mut_binary']
selected_genes = params['selected_genes']
combine_type = params['combine_type']
use_coding_genes_only = params['use_coding_genes_only']
binary_response = params['binary_response']

tcga_data_path = join(params['data_dir'], 'TCGA', 'prepared')
tcga_processed_path = join(params['data_dir'], 'TCGA', 'processed')

## 导入数据

In [7]:
if exists(join(tcga_processed_path, 'cnv_allgenes.csv')) and \
    exists(join(tcga_processed_path, 'methylation_allgenes.csv')) and \
        exists(join(tcga_processed_path, 'mut_allgenes.csv')) and \
            exists(join(tcga_processed_path, 'drug_target_allgenes.csv')) and \
                exists(join(tcga_processed_path, 'response.csv')):
    logger.info('Loading processed data...')
    recreate = False
    response_df = pd.read_csv(join(tcga_processed_path, 'response.csv'))
    cnv_allgenes_df = pd.read_csv(join(tcga_processed_path, 'cnv_allgenes.csv'), index_col=0)
    methylation_allgenes_df = pd.read_csv(join(tcga_processed_path, 'methylation_allgenes.csv'), index_col=0)
    mut_allgenes_df = pd.read_csv(join(tcga_processed_path, 'mut_allgenes.csv'), index_col=0)
    drug_target_allgenes_df = pd.read_csv(join(tcga_processed_path, 'drug_target_allgenes.csv'), index_col=0)
else:
    logger.info('Processing data...')
    recreate = True
    if not exists(tcga_processed_path):
        os.makedirs(tcga_processed_path)

Loading processed data...


In [8]:
'''
 bcr_patient_barcode
 drug_drugbankid
 drug_response
'''
response_df

,bcr_patient_barcode,drug_drugbankid,drug_response
0,TCGA-05-4384,DB00515,1
1,TCGA-05-4384,DB00361,1
2,TCGA-05-4384,DB00399,1
3,TCGA-05-4390,DB00515,3
4,TCGA-05-4390,DB00361,3
...,...,...,...
2760,TCGA-ZM-AA05,DB00290,3
2761,TCGA-ZM-AA05,DB00773,3
2762,TCGA-ZM-AA06,DB00515,3
2763,TCGA-ZM-AA06,DB00773,3


In [9]:
'''
 行：Patient ID
 列：基因
 数值：
'''
cnv_allgenes_df

,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAAS,AACS,AADAC,AADACL2,...,ZW10,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
TCGA-HD-7229,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-56-8629,1.0,-1.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,2.0,...,-1.0,-1.0,1.0,1.0,1.0,0.0,0.0,-1.0,-1.0,0.0
TCGA-YB-A89D,1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
TCGA-NH-A8F7,1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0
TCGA-CH-5788,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-VQ-A8PP,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
TCGA-HU-8243,-1.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,-1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
TCGA-E9-A227,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-QK-A6IH,-1.0,-1.0,0.0,0.0,-1.0,1.0,0.0,0.0,1.0,1.0,...,-1.0,-1.0,2.0,2.0,-1.0,0.0,0.0,0.0,-1.0,0.0


In [10]:
'''
 行：Patient ID
 列：基因
 数值：
'''
methylation_allgenes_df

,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAAS,AACS,AADAC,AADACL2,...,ZW10,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
TCGA-05-4384,0.492546,0.000000,0.580548,0.671844,0.000000,0.549193,0.069873,0.624847,0.806474,0.809977,...,0.170782,0.079762,0.290134,0.280279,0.257494,0.424378,0.531178,0.223062,0.711194,0.125439
TCGA-05-4390,0.434417,0.411008,0.706283,0.763402,0.464110,0.567353,0.069067,0.677475,0.839674,0.837718,...,0.187792,0.072077,0.492744,0.485045,0.265404,0.656241,0.531991,0.250517,0.745432,0.119650
TCGA-05-4427,0.597874,0.381608,0.428278,0.757958,0.471150,0.318778,0.080753,0.675895,0.805386,0.839768,...,0.199415,0.068610,0.450765,0.358939,0.228729,0.424364,0.527538,0.224421,0.786797,0.083326
TCGA-05-5425,0.534171,0.550126,0.756780,0.752876,0.464628,0.496587,0.066923,0.615749,0.807411,0.831873,...,0.197788,0.051061,0.326915,0.299536,0.248874,0.486545,0.507580,0.244801,0.727179,0.119705
TCGA-05-5428,0.588511,0.568327,0.634751,0.719180,0.512581,0.539810,0.079286,0.671740,0.809983,0.894442,...,0.198559,0.065721,0.342730,0.307928,0.263013,0.403131,0.456163,0.231916,0.771577,0.116002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-ZF-AA5H,0.413401,0.377015,0.428972,0.557856,0.454985,0.371461,0.048034,0.648732,0.673151,0.707070,...,0.154501,0.042690,0.444118,0.425777,0.257199,0.564667,0.501892,0.178870,0.775796,0.100436
TCGA-ZH-A8Y4,0.560912,0.000000,0.822911,0.889592,0.502357,0.540746,0.048755,0.674775,0.864024,0.938483,...,0.189635,0.047034,0.272814,0.161929,0.000000,0.799652,0.517159,0.218249,0.740945,0.100872
TCGA-ZH-A8Y5,0.492480,0.000000,0.861700,0.772864,0.491157,0.287115,0.047480,0.651731,0.455166,0.912972,...,0.179390,0.028179,0.198105,0.182938,0.268379,0.750686,0.519646,0.221281,0.826975,0.106509
TCGA-ZM-AA05,0.423666,0.587446,0.414069,0.426657,0.286682,0.353664,0.043583,0.405505,0.579642,0.642413,...,0.152012,0.030111,0.185568,0.222353,0.166864,0.363278,0.359201,0.147294,0.445972,0.082287


In [11]:
'''
 行：Patient ID
 列：基因
 数值：0/1，Patient 对应的基因突变
'''
mut_allgenes_df

,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAAS,AACS,AADAC,AADACL2,...,ZW10,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
TCGA-05-4384,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-05-4390,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-05-4398,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-05-4402,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-05-4427,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-ZF-AA5H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-ZH-A8Y4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
TCGA-ZH-A8Y5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-ZM-AA05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
'''
 行：药物
 列：基因
 数值：0/1，药物对应的 target gene
'''
drug_target_allgenes_df

,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAAS,AACS,AADAC,AADACL2,...,ZW10,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
DB00002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DB00007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DB00014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DB00016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DB00019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DB09256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DB11689,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DB11919,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DB12200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
logger.info(f'The shape of cnv_allgenes_df {cnv_allgenes_df.shape}')
logger.info(f'The shape of methylation_allgenes_df {methylation_allgenes_df.shape}')
logger.info(f'The shape of mutation_allgenes_df {mut_allgenes_df.shape}')
logger.info(f'The shape of drug_target_allgenes_df {drug_target_allgenes_df.shape}')

logger.info('{} samples and {} drugs in {} records.'.format(
    len(set(response_df['bcr_patient_barcode'])), len(set(response_df['drug_drugbankid'])), len(response_df)))

genes = list(cnv_allgenes_df.columns)
response_array = response_df.values.tolist()

The shape of cnv_allgenes_df (1441, 16307)
The shape of methylation_allgenes_df (1357, 16307)
The shape of mutation_allgenes_df (1394, 16307)
The shape of drug_target_allgenes_df (119, 16307)
1265 samples and 112 drugs in 2765 records.


In [14]:
genes

['A1BG',
 'A1CF',
 'A2M',
 'A2ML1',
 'A4GALT',
 'A4GNT',
 'AAAS',
 'AACS',
 'AADAC',
 'AADACL2',
 'AADACL3',
 'AADACL4',
 'AADAT',
 'AAGAB',
 'AAK1',
 'AAMP',
 'AANAT',
 'AARS',
 'AARS2',
 'AARSD1',
 'AASDH',
 'AASDHPPT',
 'AASS',
 'AATF',
 'AATK',
 'ABAT',
 'ABCA1',
 'ABCA10',
 'ABCA12',
 'ABCA13',
 'ABCA2',
 'ABCA3',
 'ABCA4',
 'ABCA5',
 'ABCA6',
 'ABCA7',
 'ABCA8',
 'ABCA9',
 'ABCB1',
 'ABCB10',
 'ABCB11',
 'ABCB4',
 'ABCB5',
 'ABCB6',
 'ABCB7',
 'ABCB8',
 'ABCB9',
 'ABCC1',
 'ABCC10',
 'ABCC11',
 'ABCC12',
 'ABCC2',
 'ABCC3',
 'ABCC4',
 'ABCC5',
 'ABCC6',
 'ABCC8',
 'ABCC9',
 'ABCD1',
 'ABCD2',
 'ABCD3',
 'ABCD4',
 'ABCE1',
 'ABCF1',
 'ABCF2',
 'ABCF3',
 'ABCG1',
 'ABCG2',
 'ABCG4',
 'ABCG5',
 'ABCG8',
 'ABHD1',
 'ABHD10',
 'ABHD11',
 'ABHD12',
 'ABHD12B',
 'ABHD13',
 'ABHD14A',
 'ABHD14B',
 'ABHD15',
 'ABHD2',
 'ABHD3',
 'ABHD4',
 'ABHD5',
 'ABHD6',
 'ABHD8',
 'ABI1',
 'ABI2',
 'ABI3',
 'ABI3BP',
 'ABL1',
 'ABL2',
 'ABLIM1',
 'ABLIM2',
 'ABLIM3',
 'ABO',
 'ABR',
 'ABRA',
 'ABT1',


In [15]:
response_array

[['TCGA-05-4384', 'DB00515', 1],
 ['TCGA-05-4384', 'DB00361', 1],
 ['TCGA-05-4384', 'DB00399', 1],
 ['TCGA-05-4390', 'DB00515', 3],
 ['TCGA-05-4390', 'DB00361', 3],
 ['TCGA-05-4427', 'DB00361', 3],
 ['TCGA-05-4427', 'DB00515', 3],
 ['TCGA-05-5425', 'DB01229', 1],
 ['TCGA-05-5425', 'DB00958', 1],
 ['TCGA-05-5425', 'DB00317', 1],
 ['TCGA-05-5428', 'DB00515', 3],
 ['TCGA-05-5428', 'DB00361', 3],
 ['TCGA-06-1806', 'DB07232', 1],
 ['TCGA-06-1806', 'DB08875', 1],
 ['TCGA-06-A7TK', 'DB01168', 1],
 ['TCGA-06-A7TK', 'DB01206', 1],
 ['TCGA-06-A7TL', 'DB12978', 0],
 ['TCGA-13-A5FT', 'DB01229', 3],
 ['TCGA-13-A5FT', 'DB00515', 3],
 ['TCGA-13-A5FU', 'DB01229', 2],
 ['TCGA-13-A5FU', 'DB00958', 2],
 ['TCGA-19-5950', 'DB08828', 1],
 ['TCGA-19-5956', 'DB00112', 0],
 ['TCGA-19-5959', 'DB00112', 1],
 ['TCGA-19-5960', 'DB00112', 0],
 ['TCGA-26-A7UX', 'DB01206', 0],
 ['TCGA-26-A7UX', 'DB00112', 0],
 ['TCGA-29-A5NZ', 'DB00441', 1],
 ['TCGA-29-A5NZ', 'DB01229', 3],
 ['TCGA-29-A5NZ', 'DB00958', 2],
 ['TCGA-29

## 训练集、验证集、测试集划分

In [16]:
def _get_class_num(df, data='train'):
    logger.info(f'The number of samples for each response value in {data}:')
    for response_value in set(df['drug_response']):
        logger.info(f'{response_value}: {len(df[df["drug_response"]==response_value])}')

In [17]:
binary_response

False

In [18]:
seed = 1
split_dir = join(data_path, 'TCGA', 'split')
if not os.path.exists(split_dir):
    os.makedirs(split_dir)
if binary_response == True:
    train_file_name = join(split_dir, 'training_set_binary_seed_'+str(seed)+'.csv')
    valid_file_name = join(split_dir, 'validation_set_binary_seed_'+str(seed)+'.csv')
    test_file_name = join(split_dir, 'test_set_binary_seed_'+str(seed)+'.csv')
    logger.info('Convert response to binary: clinical response value larger than (or equal) 2 -> 1.')
    response_df['drug_response'] = response_df['drug_response'].apply(lambda x: 1 if x >= 2 else 0)
    _get_class_num(response_df, data='all')
else:
    # count the number of samples for each response value
    _get_class_num(response_df, data='all')
    logger.info('\n')
    train_file_name = join(split_dir, 'training_set_seed_'+str(seed)+'.csv')
    valid_file_name = join(split_dir, 'valid_set_seed_'+str(seed)+'.csv')
    test_file_name = join(split_dir, 'test_set_seed_'+str(seed)+'.csv')

if not exists(train_file_name) or recreate:
    logger.info(f'Spliting the dataset with seed {seed}')
    sample_train, sample_test = train_test_split(response_df, test_size=0.2)
    sample_test, sample_valid = train_test_split(sample_test, test_size=0.5)
    sample_train.to_csv(train_file_name, index=False)
    sample_valid.to_csv(valid_file_name, index=False)
    sample_test.to_csv(test_file_name, index=False)
else:
    logger.info(f'Loading the split index with seed {seed}.')
    sample_train = pd.read_csv(train_file_name)
    sample_valid = pd.read_csv(valid_file_name)
    sample_test = pd.read_csv(test_file_name)

_get_class_num(sample_train, data='train')
_get_class_num(sample_valid, data='valid')
_get_class_num(sample_test, data='test')

respone_train_array = sample_train.values.tolist()
respone_valid_array = sample_valid.values.tolist()
respone_test_array = sample_test.values.tolist()

The number of samples for each response value in all:
0: 276
1: 794
2: 223
3: 1472


Loading the split index with seed 1.
The number of samples for each response value in train:
0: 216
1: 636
2: 181
3: 1179
The number of samples for each response value in valid:
0: 34
1: 70
2: 19
3: 154
The number of samples for each response value in test:
0: 26
1: 88
2: 23
3: 139


In [19]:
K = 5
Kfold_seed_path = Path(data_path).joinpath('TCGA', 'Kfold-{}_seed-{}'.format(K, seed))
if not exists(Kfold_seed_path):
    os.makedirs(Kfold_seed_path)
if binary_response == True:
    logger.info('Convert response to binary: clinical response value larger than (or equal) 2 -> 1.')
    response_df['drug_response'] = response_df['drug_response'].apply(lambda x: 1 if x >= 2 else 0)
    _get_class_num(response_df, data='all')
    filename_prefix = 'binary_'
else:
    _get_class_num(response_df, data='all')
    filename_prefix = ''

if not exists(Kfold_seed_path.joinpath(filename_prefix + 'training_K0.csv')) or recreate == True:
    kf = KFold(n_splits=K, shuffle=True, random_state=seed)
    for i, (train_index, test_index) in enumerate(kf.split(response_df)):
        sample_train = response_df.iloc[train_index]
        sample_test = response_df.iloc[test_index]
        sample_train.to_csv(Kfold_seed_path.joinpath(filename_prefix + 'training_K{}.csv'.format(i)), index=False)
        sample_test.to_csv(Kfold_seed_path.joinpath(filename_prefix + 'test_K{}.csv'.format(i)), index=False)

The number of samples for each response value in all:
0: 276
1: 794
2: 223
3: 1472
